<a href="https://colab.research.google.com/github/sanaaria/Master-thesis/blob/main/hgcn_better%20trsin%2Ctest_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow==2.18.1
!pip install -q torch==2.4.0 dgl==2.1.0
!pip install -q ampligraph==2.0.0 scikit-learn pandas numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.6/615.6 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 60.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.18.1 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.18.1 which is incompatible.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.18.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 826.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [2]:
import os
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Tuple, Dict

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import (roc_auc_score, average_precision_score,
                             f1_score, matthews_corrcoef, precision_recall_curve)

import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.nn import HeteroGraphConv, GraphConv

# AmpliGraph
from ampligraph.latent_features import ScoringBasedEmbeddingModel
import ampligraph.latent_features.loss_functions as lfs
from ampligraph.latent_features.loss_functions import get as get_loss
from ampligraph.latent_features.regularizers import get as get_regularizer
import tensorflow as tf

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


ModuleNotFoundError: No module named 'torchdata.datapipes'

In [ ]:
# -------------------- Config --------------------
@dataclass
class CFG:
    seed: int = 42
    # paths for your triples (positive edges). REQUIRED: columns=['head','relation','tail']
    triples_csv: str = "data/yamanishi_08/dt_all_08.txt"  # TSV or CSV (auto-detected below)
    triples_sep: str = "\t"  # '\t' for .txt files, ',' for CSV
    # optional side features:
    drug_fp_csv: str = "data/yamanishi_08/791drug_struc.csv"     # columns: ['drug_id', f1, f2, ...]
    pro_desc_csv: str = "data/yamanishi_08/989proseq.csv"        # columns: ['pro_id','pro_ids','seq']  (we'll ignore text; just ids)
    drug_feats_txt: str = "data/yamanishi_08/morganfp.txt"       # numeric CSV-like
    pro_feats_txt: str  = "data/yamanishi_08/pro_ctd.txt"        # numeric CSV-like

    # split ratios:
    train_ratio: float = 0.8
    valid_ratio: float = 0.1  # test = 1 - train - valid

    # negative sampling:
    num_neg_per_pos_train: int = 1
    num_neg_per_pos_eval: int  = 1

    # KG embedding (AmpliGraph):
    k: int = 200              # embedding dimension
    eta: int = 10             # negatives per positive (internal to AmpliGraph)
    epochs_kg: int = 50       # train AmpliGraph epochs
    batches_count: int = 1000
    lr_kg: float = 1e-3
    reg_lambda: float = 1e-5
    early_stop_patience_kg: int = 5

    # Node feature post-processing
    use_pca: bool = True
    pca_dim: int = 200

    # HGCN model:
    g_hidden: int = 256
    g_dropout: float = 0.5
    lr_hgcn: float = 1e-3
    weight_decay: float = 1e-4
    epochs_hgcn: int = 30

    # device:
    device: str = "cuda" if torch.cuda.is_available() else "cpu"

cfg = CFG()


In [ ]:


# -------------------- Utils --------------------
def set_seed(seed: int):
    np.random.seed(seed)
    torch.manual_seed(seed)
    tf.random.set_seed(seed)
set_seed(cfg.seed)

def load_triples(triples_path: str, sep: str) -> pd.DataFrame:
    df = pd.read_csv(triples_path, sep=sep, header=None, names=['head','relation','tail'])
    # remove exact duplicates
    df = df.drop_duplicates().reset_index(drop=True)
    return df

def label_encode_nodes_and_relations(df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict, Dict]:
    """Returns df_idx with integer ids, plus node_index and rel_index dicts."""
    nodes = pd.Index(pd.concat([df['head'], df['tail']]).unique())
    node2id = {n:i for i,n in enumerate(nodes)}
    rels = pd.Index(df['relation'].unique())
    rel2id = {r:i for i,r in enumerate(rels)}
    df_idx = df.assign(
        head=df['head'].map(node2id),
        tail=df['tail'].map(node2id),
        relation=df['relation'].map(rel2id)
    )
    return df_idx, node2id, rel2id

def pairwise_split(df_pos_idx: pd.DataFrame, seed: int) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Pair-wise random split over positive edges."""
    train_pos, tmp = train_test_split(df_pos_idx, test_size=(1-cfg.train_ratio), random_state=seed, shuffle=True)
    valid_ratio_adj = cfg.valid_ratio / (1 - cfg.train_ratio)
    valid_pos, test_pos = train_test_split(tmp, test_size=(1-valid_ratio_adj), random_state=seed+1, shuffle=True)
    return train_pos.reset_index(drop=True), valid_pos.reset_index(drop=True), test_pos.reset_index(drop=True)

def negative_sample(all_nodes: np.ndarray, pos_df: pd.DataFrame, num_neg: int, seed: int) -> pd.DataFrame:
    """Simple head/tail corruption avoiding trivial leaks (within the given pos_df scope)."""
    rng = np.random.default_rng(seed)
    pos_set = set(map(tuple, pos_df[['head','relation','tail']].values))
    neg_rows = []
    for _ in range(len(pos_df)*num_neg):
        h, r, t = pos_df.sample(1, random_state=rng.integers(1e9)).values[0]
        if rng.random() > 0.5:
            new_h = int(rng.choice(all_nodes))
            trip = (new_h, r, t)
        else:
            new_t = int(rng.choice(all_nodes))
            trip = (h, r, new_t)
        if trip not in pos_set:
            neg_rows.append(trip)
    neg_df = pd.DataFrame(neg_rows, columns=['head','relation','tail'])
    return neg_df

def build_train_graph(train_pos: pd.DataFrame, num_nodes: int, num_relations: int) -> dgl.DGLHeteroGraph:
    # Build heterograph: one node type, multiple relation types
    rel_dict = {}
    train_pos_np = train_pos[['head','relation','tail']].values
    for rel_id in range(num_relations):
        mask = (train_pos_np[:,1] == rel_id)
        if mask.sum() == 0:
            # empty relation; create an empty edge list
            rel_dict[('node','r'+str(rel_id),'node')] = (np.array([],dtype=np.int64), np.array([],dtype=np.int64))
        else:
            edges = train_pos_np[mask]
            src = edges[:,0].astype(np.int64)
            dst = edges[:,2].astype(np.int64)
            rel_dict[('node','r'+str(rel_id),'node')] = (src, dst)
    g = dgl.heterograph(rel_dict, num_nodes_dict={'node': num_nodes})
    return g

# -------------------- KG Embeddings (AmpliGraph) --------------------
def fit_ampligraph_embeddings(train_pos: pd.DataFrame, num_nodes: int, num_rels: int) -> Tuple[np.ndarray, np.ndarray]:
    """Fit DistMult on train positives only; return entity and relation embeddings arrays."""
    # Prepare triples for AmpliGraph: shape (n,3) with int ids is fine
    triples = train_pos[['head','relation','tail']].values.astype(np.int64)

    optim = tf.keras.optimizers.Adam(learning_rate=cfg.lr_kg)
    loss = get_loss('pairwise', {'margin': 0.5})
    regularizer = get_regularizer('LP', {'p': 2, 'lambda': cfg.reg_lambda})

    model = ScoringBasedEmbeddingModel(
        k=cfg.k, eta=cfg.eta, scoring_type="DistMult", seed=cfg.seed
    )
    model.compile(optimizer=optim, loss=loss, entity_relation_regularizer=regularizer)

    early = tf.keras.callbacks.EarlyStopping(
        monitor="loss", patience=cfg.early_stop_patience_kg, restore_best_weights=True
    )

    model.fit(triples, batch_size=cfg.batches_count, epochs=cfg.epochs_kg, callbacks=[early])

    # get entity & relation embeddings by id range:
    ent_ids = np.arange(num_nodes, dtype=np.int64)
    rel_ids = np.arange(num_rels, dtype=np.int64)
    ent_emb = model.get_embeddings(ent_ids, embedding_type='e')  # (num_nodes, k)
    rel_emb = model.get_embeddings(rel_ids, embedding_type='r')  # (num_rels, k)
    return ent_emb, rel_emb

# -------------------- Node Features --------------------
def optional_side_features(node2id: Dict, drug_fp_csv: str, pro_desc_csv: str,
                           drug_feats_txt: str, pro_feats_txt: str) -> np.ndarray:
    """
    Optional: concatenate drug/protein side features. If files missing, return zeros.
    Assumes drug ids and protein ids in your triples are numeric ids after encoding.
    You can adapt this part to map your real-world drug/protein ids -> node indices.
    """
    try:
        fp_id = pd.read_csv(drug_fp_csv)['drug_id']
        drug_feats = np.loadtxt(drug_feats_txt, delimiter=',')
        df_drug = pd.concat([fp_id, pd.DataFrame(drug_feats)], axis=1)
        df_drug.columns = ['drug_id'] + [f'df_{i}' for i in range(drug_feats.shape[1])]
    except Exception:
        df_drug = None

    try:
        df_proseq = pd.read_csv(pro_desc_csv)  # ['pro_id','pro_ids','seq']
        pro_feats = np.loadtxt(pro_feats_txt, delimiter=',')
        df_pro = pd.concat([df_proseq['pro_id'], pd.DataFrame(pro_feats)], axis=1)
        df_pro.columns = ['pro_id'] + [f'pf_{i}' for i in range(pro_feats.shape[1])]
    except Exception:
        df_pro = None

    num_nodes = len(node2id)
    # Fallback zeros
    if (df_drug is None) and (df_pro is None):
        return np.zeros((num_nodes, 0), dtype=np.float32)

    # Minimal placeholder mapping: here we can't know which node is drug/protein in generic CSV.
    # In your data model, you likely have separate namespaces (drug_*, prot_*). Adapt if available.
    # For safety, return zeros unless you wire exact mapping.
    return np.zeros((num_nodes, 0), dtype=np.float32)

def build_node_features(ent_emb: np.ndarray,
                        side_feats: np.ndarray,
                        use_pca=True, pca_dim=200) -> Tuple[np.ndarray, MinMaxScaler, PCA]:
    feats = ent_emb if side_feats.shape[1] == 0 else np.concatenate([ent_emb, side_feats], axis=1)
    scaler = MinMaxScaler()
    feats_scaled = scaler.fit_transform(feats)
    if use_pca and feats_scaled.shape[1] > pca_dim:
        pca = PCA(n_components=pca_dim, random_state=cfg.seed)
        feats_p = pca.fit_transform(feats_scaled)
    else:
        pca = None
        feats_p = feats_scaled
    return feats_p.astype(np.float32), scaler, pca

# -------------------- HGCN Model (RGCN-like) --------------------
class RGCNLayer(nn.Module):
    def __init__(self, in_dim, out_dim, relations):
        super().__init__()
        self.conv = HeteroGraphConv(
            {('node', f'r{i}', 'node'): GraphConv(in_dim, out_dim) for i in range(relations)},
            aggregate='sum'
        )

    def forward(self, g, h_dict):
        h = self.conv(g, h_dict)
        return {k: F.relu(v) for k, v in h.items()}

class HGCNLinkPredictor(nn.Module):
    def __init__(self, in_dim, hidden_dim, relations, dropout=0.5):
        super().__init__()
        self.layer1 = RGCNLayer(in_dim, hidden_dim, relations)
        self.layer2 = RGCNLayer(hidden_dim, hidden_dim, relations)
        self.dropout = nn.Dropout(dropout)
        # edge scorer: simple bilinear (could be MLP)
        self.scorer = nn.Bilinear(hidden_dim, hidden_dim, 1)

    def forward_node(self, g, x):
        h = {'node': x}
        h = self.layer1(g, h)
        h = self.layer2(g, h)
        z = self.dropout(h['node'])
        return z  # node embeddings

    def score_edges(self, z, heads, tails):
        # z: [N, H], heads/tails: [B]
        return self.scorer(z[heads], z[tails]).squeeze(-1)

# -------------------- Training & Eval --------------------
def batches(edges_pos: pd.DataFrame, edges_neg: pd.DataFrame, batch_size=4096, shuffle=True, device='cpu'):
    pos_arr = edges_pos[['head','tail']].values
    neg_arr = edges_neg[['head','tail']].values
    labels_pos = np.ones(len(pos_arr), dtype=np.float32)
    labels_neg = np.zeros(len(neg_arr), dtype=np.float32)

    X = np.vstack([pos_arr, neg_arr])
    y = np.concatenate([labels_pos, labels_neg])

    idx = np.arange(len(y))
    if shuffle:
        np.random.shuffle(idx)
    for i in range(0, len(y), batch_size):
        j = idx[i:i+batch_size]
        heads = torch.tensor(X[j,0], dtype=torch.long, device=device)
        tails = torch.tensor(X[j,1], dtype=torch.long, device=device)
        labels = torch.tensor(y[j], dtype=torch.float32, device=device)
        yield heads, tails, labels

def train_hgcn(g, node_feats, train_pos, train_neg, valid_pos, valid_neg, num_relations, device='cpu'):
    model = HGCNLinkPredictor(
        in_dim=node_feats.shape[1],
        hidden_dim=cfg.g_hidden,
        relations=num_relations,
        dropout=cfg.g_dropout
    ).to(device)

    opt = torch.optim.Adam(model.parameters(), lr=cfg.lr_hgcn, weight_decay=cfg.weight_decay)
    bce = nn.BCEWithLogitsLoss()

    x = torch.tensor(node_feats, dtype=torch.float32, device=device)

    best_val = -1.0
    best_state = None

    for epoch in range(cfg.epochs_hgcn):
        model.train()
        z = model.forward_node(g, x)
        total_loss = 0.0
        for h, t, y in batches(train_pos, train_neg, device=device):
            opt.zero_grad()
            logits = model.score_edges(z, h, t)
            loss = bce(logits, y)
            loss.backward()
            opt.step()
            total_loss += loss.item()

        # validation
        model.eval()
        with torch.no_grad():
            z_val = model.forward_node(g, x)
            # scores
            def predict(pos, neg):
                hp = torch.tensor(pos[['head','tail']].values[:,0], dtype=torch.long, device=device)
                tp = torch.tensor(pos[['head','tail']].values[:,1], dtype=torch.long, device=device)
                hn = torch.tensor(neg[['head','tail']].values[:,0], dtype=torch.long, device=device)
                tn = torch.tensor(neg[['head','tail']].values[:,1], dtype=torch.long, device=device)
                sp = torch.sigmoid(model.score_edges(z_val, hp, tp)).cpu().numpy()
                sn = torch.sigmoid(model.score_edges(z_val, hn, tn)).cpu().numpy()
                y_true = np.concatenate([np.ones_like(sp), np.zeros_like(sn)])
                y_pred = np.concatenate([sp, sn])
                return y_true, y_pred

            yv, pv = predict(valid_pos, valid_neg)
            auc = roc_auc_score(yv, pv)
            aupr = average_precision_score(yv, pv)

        if auc > best_val:
            best_val = auc
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}

        print(f"[Epoch {epoch+1:02d}] loss={total_loss:.4f}  val_auc={auc:.4f}  val_aupr={aupr:.4f}")

    # load best
    if best_state is not None:
        model.load_state_dict({k: v.to(device) for k, v in best_state.items()})
    model.eval()
    return model

def evaluate(model, g, node_feats, test_pos, test_neg, device='cpu'):
    x = torch.tensor(node_feats, dtype=torch.float32, device=device)
    with torch.no_grad():
        z = model.forward_node(g, x)
        hp = torch.tensor(test_pos[['head','tail']].values[:,0], dtype=torch.long, device=device)
        tp = torch.tensor(test_pos[['head','tail']].values[:,1], dtype=torch.long, device=device)
        hn = torch.tensor(test_neg[['head','tail']].values[:,0], dtype=torch.long, device=device)
        tn = torch.tensor(test_neg[['head','tail']].values[:,1], dtype=torch.long, device=device)

        sp = torch.sigmoid(model.score_edges(z, hp, tp)).cpu().numpy()
        sn = torch.sigmoid(model.score_edges(z, hn, tn)).cpu().numpy()

    y_true = np.concatenate([np.ones_like(sp), np.zeros_like(sn)])
    y_pred = np.concatenate([sp, sn])

    auc = roc_auc_score(y_true, y_pred)
    aupr = average_precision_score(y_true, y_pred)

    # Choose threshold at best F1 (from PR curve)
    prec, rec, thr = precision_recall_curve(y_true, y_pred)
    f1_arr = 2*prec*rec/(prec+rec+1e-9)
    best_idx = np.nanargmax(f1_arr)
    thr_star = thr[max(0, best_idx-1)] if len(thr) else 0.5
    y_bin = (y_pred >= thr_star).astype(np.int32)

    f1 = f1_score(y_true, y_bin)
    mcc = matthews_corrcoef(y_true, y_bin)
    return dict(AUC=auc, AUPR=aupr, F1=f1, MCC=mcc, THR=thr_star)

# -------------------- Main --------------------
def main():
    print("Device:", cfg.device)
    # 1) Load positives
    df_pos_raw = load_triples(cfg.triples_csv, cfg.triples_sep)
    df_pos_idx, node2id, rel2id = label_encode_nodes_and_relations(df_pos_raw)
    num_nodes = len(node2id)
    num_rels  = len(rel2id)

    # 2) Split (pair-wise random)
    train_pos, valid_pos, test_pos = pairwise_split(df_pos_idx, cfg.seed)

    # 3) Negative sampling (separately per split; no cross-leak)
    all_nodes = np.arange(num_nodes)
    train_neg = negative_sample(all_nodes, train_pos[['head','relation','tail']], cfg.num_neg_per_pos_train, cfg.seed+10)
    valid_neg = negative_sample(all_nodes, valid_pos[['head','relation','tail']], cfg.num_neg_per_pos_eval,  cfg.seed+11)
    test_neg  = negative_sample(all_nodes, test_pos[['head','relation','tail']],  cfg.num_neg_per_pos_eval,  cfg.seed+12)

    # 4) AmpliGraph embeddings (fit on TRAIN positives only)
    ent_emb, rel_emb = fit_ampligraph_embeddings(train_pos[['head','relation','tail']], num_nodes, num_rels)

    # 5) Optional side features (zeros by default; wire mapping if you want)
    side_feats = optional_side_features(node2id, cfg.drug_fp_csv, cfg.pro_desc_csv,
                                        cfg.drug_feats_txt, cfg.pro_feats_txt)

    node_feats, scaler, pca = build_node_features(ent_emb, side_feats, use_pca=cfg.use_pca, pca_dim=cfg.pca_dim)

    # 6) Build TRAIN graph only (to avoid leakage)
    g_train = build_train_graph(train_pos, num_nodes, num_rels)
    g_train = dgl.add_self_loop(g_train)
    g_train = g_train.to(cfg.device)

    # 7) Train HGCN
    model = train_hgcn(g_train, node_feats, train_pos, train_neg, valid_pos, valid_neg,
                       num_relations=num_rels, device=cfg.device)

    # 8) Evaluate on TEST (scoring edges; graph remains train-only)
    metrics = evaluate(model, g_train, node_feats, test_pos, test_neg, device=cfg.device)
    print("\n==== TEST METRICS ====")
    for k,v in metrics.items():
        if isinstance(v, float):
            print(f"{k}: {v:.4f}")
        else:
            print(f"{k}: {v}")

if __name__ == "__main__":
    main()
